In [1]:
import pandas as pd

def parse_gedcom(file_path):
    individuals = {}
    current_individual = None
    current_individual_data = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith('0 @I'):
                if current_individual is not None:
                    individuals[current_individual] = current_individual_data
                    current_individual_data = {}
                current_individual = line.split('@')[1]
            elif line.startswith('1'):
                current_tag = line.split(' ')[1]
                value = line.split(' ')[2:]
                current_individual_data[current_tag] = value
                
            elif line.startswith('2'):
                add_tag = line.split(' ')[1]
                current_tag = current_tag + add_tag
                value = line.split(' ')[2:]
                current_individual_data[current_tag] = value              
                
            else:
                continue

        if current_individual is not None:
            individuals[current_individual] = current_individual_data

    return individuals

individuals = parse_gedcom('BothHarmonSniderTree.ged')

individual_data = []
for individual_id, individual in individuals.items():
    data = {'ID': individual_id}
    for tag, values in individual.items():
        data[tag] = ' '.join(values)
    individual_data.append(data)

individual_df = pd.DataFrame(individual_data)


In [12]:
import pandas as pd

# Create a sample DataFrame
data = {'Date': ['12 Jan 2022', 'Feb 2023', '2024', None, '15 Mar 2021']}
df = pd.DataFrame(data)

# Create a new column to store the cleaned dates
df['Cleaned_Date'] = pd.NaT  # Initialize with NaT (Not a Time)

# Convert most dates to datetime format
df['Cleaned_Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Handle remaining dates that are not in the correct format
def clean_date(date):
    if pd.isnull(date):  # Handle missing dates
        return pd.NaT
    elif isinstance(date, pd.Timestamp):  # Handle Timestamp objects
        return date
    elif len(str(date)) == 4:  # Handle YYYY format
        return pd.to_datetime(str(date) + '-01-01')
    elif len(str(date)) == 7:  # Handle MMM YYYY format
        return pd.to_datetime(str(date) + '-01')
    else:
        return pd.to_datetime(date)

df['Cleaned_Date'] = df['Cleaned_Date'].apply(clean_date)

# Convert the cleaned dates to YYYY-MM-DD format
df['Cleaned_Date'] = df['Cleaned_Date'].dt.strftime('%Y-%m-%d')

print(df)

          Date Cleaned_Date
0  12 Jan 2022   2022-01-12
1     Feb 2023          NaN
2         2024          NaN
3         None          NaN
4  15 Mar 2021   2021-03-15
